In [19]:
import torch
import os
from firstLearn import CURRENT_DIRECTORY, folderMustExist, ImageRemembererNet,genPilImage

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [25]:
net = ImageRemembererNet(500, 6)
net.to(device)

netOld = ImageRemembererNet(500, 6)
netOld.to(device)

netNew = ImageRemembererNet(500, 6)
netNew.to(device)

netOld.load(os.path.join(CURRENT_DIRECTORY, "models-first", "175.model"))
old_state_dict = netOld.state_dict().copy()

netNew.load(os.path.join(CURRENT_DIRECTORY, "models-second", "8.model"))
new_state_dict = netNew.state_dict().copy()

In [26]:
output_path = os.path.join(CURRENT_DIRECTORY, "steps")
steps_count = 100


height = 100
width = 100
X = torch.FloatTensor([(x,y) for x in range(height) for y in range(width)]).to(device)

In [27]:
keys = old_state_dict.keys()
current_state_dict = old_state_dict.copy()
step_state_dict = new_state_dict.copy()
for key in map(str, keys):
    step_state_dict[key] = (step_state_dict[key] - old_state_dict[key]) / steps_count


In [28]:
net.load_state_dict(current_state_dict)
preds = net.inference(X)
genPilImage(height, width, preds.cpu().detach().numpy()).save(os.path.join(output_path,f"{0}.png"))
for i in range(steps_count):
    for key in map(str, keys):
        current_state_dict[key] += step_state_dict[key]
    net.load_state_dict(current_state_dict)
    preds = net.inference(X)
    genPilImage(height, width, preds.cpu().detach().numpy()).save(os.path.join(output_path,f"{i+1}.png"))